<a href="https://colab.research.google.com/github/anslemhuncho/MACHINE-LEARNING-PROJECT/blob/anslem/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

convert images into matplot files

In [2]:
from google.colab import drive
import os
import scipy.io
import numpy as np
from PIL import Image

def convert_images_to_mat(root_folder, output_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image_path = os.path.join(root, file)
                image = Image.open(image_path)
                image_data = np.array(image)

                # Create output folder if it doesn't exist
                os.makedirs(output_folder, exist_ok=True)

                # Save image data to .mat file
                file_name = os.path.splitext(file)[0] + '.mat'
                output_path = os.path.join(output_folder, file_name)
                scipy.io.savemat(output_path, {'image_data': image_data})

# Example usage
root_folder = '/content/drive/MyDrive/shangai tech image dataset/shangai tech image dataset'
output_folder = '/content/drive/MyDrive/normalised data/processed data'
convert_images_to_mat(root_folder, output_folder)



mount drive in colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


compile the model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

# Define precision and recall as custom metrics
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    actual_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (actual_positives + K.epsilon())
    return recall

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(254, 254, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model with additional metrics (precision and recall)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precision, recall])

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/shangai tech image dataset/shangai tech image dataset',
    target_size=(254, 254),
    batch_size=250,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/shangai tech image dataset/shangai tech image dataset',
    target_size=(254, 254),
    batch_size=250,
    class_mode='categorical',
    subset='validation'
)

# Train the model
try:
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator
    )
except Exception as e:
    print("Error:", e)
    # Print additional debugging information or traceback if needed


Found 216 images belonging to 2 classes.
Found 53 images belonging to 2 classes.
Epoch 1/10
1/1 [==============================] - 35s 35s/step - loss: 0.6633 - accuracy: 0.9306 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 34.9322 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 27s 27s/step - loss: 1.1443e-24 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 60.6573 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 30s 30s/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 82.6119 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 28s 28s/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 102.2566 - val_accuracy: 0.5000 - val_p

print predictions

In [5]:
# Load a batch of test images
test_images, test_labels = next(validation_generator)  # assuming validation_generator is set up

# Make predictions for the test images
predictions = model.predict(test_images)
#print predictions
for i in range(len(test_images)):
    prediction = predictions[i][0]  # get the probability for class 1 (crowd)
    actual_label = 'crowd' if test_labels[i].any() == 1 else 'non-crowd'  # use .any() to check if any element in the array is 1
    print(f"Image {i+1}: Predicted Probability for Crowd: {prediction:.4f}, Actual Label: {actual_label}")



2/2 [==============================] - 1s 538ms/step
Image 1: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 2: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 3: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 4: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 5: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 6: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 7: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 8: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 9: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 10: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 11: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 12: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 13: Predicted Probability for Crowd: 0.0000, Actual Label: crowd
Image 14: Predicted Probability f